In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from tqdm import tqdm
from textblob import TextBlob
from transformers import pipeline
import spacy
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import re

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Définir les en-têtes HTTP pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

# Fonction pour scraper une page de recherche et récupérer les articles
def scrape_page(url, session):
    all_articles = []  # Liste de tous les articles
    seen_articles = set()  # Ensemble pour garder trace des articles déjà ajoutés

    try:
        # Envoyer une requête HTTP pour obtenir la page
        response = session.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Trouver tous les articles pour les deux cas
            articles = soup.find_all("article", class_=["fig-profil fig-profil-mtpd", "fig-profil"])

            for article in articles:
                # Extraire le titre
                title_element = article.find("h2", class_="fig-profil-headline")
                title_link = title_element.find("a") if title_element else None
                title_text = title_link.get_text(strip=True) if title_link else "Titre non disponible"

                # Extraire l'URL de l'article
                article_url = title_link["href"] if title_link else None

                # Extraire la description
                description_element = article.find("div", class_="fig-profil-chapo")
                description_text = description_element.get_text(strip=True) if description_element else "Description non disponible"

                # Extraire la date de publication
                date_element = article.find("li", class_="fig-date-pub")
                time_element = date_element.find("time") if date_element else None
                date_text = time_element["datetime"] if time_element else "Date non disponible"

                # Ajouter l'article à la liste si non déjà vu
                if article_url and article_url not in seen_articles:
                    all_articles.append({
                        "Titre": title_text,
                        "Description": description_text,
                        "URL": article_url,
                        "Date": date_text
                    })
                    seen_articles.add(article_url)

            # Gérer le bouton "Voir plus de résultats"
            more_button = soup.find("button", class_="fh-kw__lk fh-kw__lk--more")
            if more_button:
                # Simuler un clic pour charger plus de résultats
                next_url = more_button.get("data-url")
                if next_url:
                    print(f"Bouton 'Voir plus de résultats' détecté. Scraping de {next_url}")
                    all_articles.extend(scrape_page(next_url, session))
            
            # Gérer le panneau "Mon compte" pour charger dynamiquement les articles
            login_button = soup.find("button", class_="fh-user__login fh-user__login--is-connected")
            if login_button:
                print("Bouton 'Mon compte' détecté. Articles supplémentaires peuvent être chargés.")
                # Simuler une interaction ou vérifier si plus de contenu s'affiche dynamiquement

            return all_articles

        else:
            print(f"Erreur HTTP {response.status_code} lors de l'accès à {url}")
            return []

    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return []

# URL de base de la recherche
base_url = "https://recherche.lefigaro.fr/recherche/agriculture%20bio/"

# Initialiser une session HTTP pour gérer les requêtes
with requests.Session() as session:
    # Scraper la page unique
    articles = scrape_page(base_url, session)



Bouton 'Mon compte' détecté. Articles supplémentaires peuvent être chargés.


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Configurer les options Chrome
options = Options()
options.add_argument("--headless")  # Mode sans tête
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")  # Évite les problèmes de mémoire partagée

# Initialiser le WebDriver
service = Service('/usr/local/bin/chromedriver')  # Assurez-vous que le chemin est correct
driver = webdriver.Chrome(service=service, options=options)



def scrape_page_with_selenium(base_url):
    all_articles = []  # Liste de tous les articles
    seen_articles = set()  # Ensemble pour éviter les doublons

    try:
        # Charger la page
        driver.get(base_url)

        while True:
            # Attendre que le bouton "Voir plus de résultats" soit présent
            try:
                more_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "js_startendeless"))
                )
                # Scroller jusqu'au bouton pour le rendre visible
                ActionChains(driver).move_to_element(more_button).perform()
                # Cliquer sur le bouton pour charger plus de résultats
                more_button.click()
                time.sleep(2)  # Attendre que les articles se chargent
            except Exception as e:
                print("Aucun bouton 'Voir plus de résultats' ou fin des articles.")
                break

        # Une fois tous les articles chargés, récupérer le contenu de la page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all("article", class_=["fig-profil fig-profil-mtpd", "fig-profil"])

        for article in articles:
            # Extraire le titre
            title_element = article.find("h2", class_="fig-profil-headline")
            title_link = title_element.find("a") if title_element else None
            title_text = title_link.get_text(strip=True) if title_link else "Titre non disponible"

            # Extraire l'URL de l'article
            article_url = title_link["href"] if title_link else None

            # Extraire la description
            description_element = article.find("div", class_="fig-profil-chapo")
            description_text = description_element.get_text(strip=True) if description_element else "Description non disponible"

            # Extraire la date de publication
            date_element = article.find("li", class_="fig-date-pub")
            time_element = date_element.find("time") if date_element else None
            date_text = time_element["datetime"] if time_element else "Date non disponible"

            # Ajouter l'article à la liste si non déjà vu
            if article_url and article_url not in seen_articles:
                all_articles.append({
                    "Titre": title_text,
                    "Description": description_text,
                    "URL": article_url,
                    "Date": date_text
                })
                seen_articles.add(article_url)

        return all_articles

    finally:
        driver.quit()  # Fermer le navigateur

# URL de base de la recherche
base_url = "https://recherche.lefigaro.fr/recherche/agriculture%20bio/"

# Appeler la fonction
articles = scrape_page_with_selenium(base_url)

# Afficher les résultats
for article in articles:
    print(article)


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [3]:
len(articles)

20